## Language Servers

By default `jupyter-lsp` does not come with any language servers preinstalled.
However, we will try to use them if they _are_ installed and we know about them
(i.e. someone contributed a full specification).

> You can disable auto-detection by configuring
> [autodetect](./Configuring.ipynb#autodetect)

You can add another language server for languages that are not listed on this
page:

- using a minimal JSON or Python
  [configuration file](./Configuring.ipynb#language_servers) (good for
  experimenting or configuring a niche server), or
- contributing a [full specification](./Contributing.ipynb#Specs) (to enable
  better integration and help other users of the same language)

The existing language servers are listed on the [official
list][lsp-implementations] and on the [community-curated list][langserver].

For the language servers in the tables below, use one of the suggested package
managers to install them: these implementations are tested to work with
`jupyter-lsp`.

[language-server]:
  https://microsoft.github.io/language-server-protocol/specification
[langserver]: https://langserver.org
[lsp-implementations]:
  https://microsoft.github.io/language-server-protocol/implementors/servers

In [ ]:
import pathlib

import IPython
from jinja2 import Template

from jupyter_lsp import LanguageServerManager

In [ ]:
mgr = LanguageServerManager(extra_node_roots=[str(pathlib.Path.cwd().parent)])

In [ ]:
mgr.init_language_servers()
servers = mgr.language_servers
servers["robotframework_ls"]["install"] = {
    "pip": "pip install robotframework-lsp",
    "conda": "conda install -c conda-forge robotframework-lsp",
}

In [ ]:
def lang_server_table(specs):
    return IPython.display.HTML(
        Template(
            """
        <table class="langservers">
            <thead>
                <tr>
                    <th>Languages</th>
                    <th>Implementation</th>
                    <th>Installation</th>
                </tr>
            </thead>
            <tbody>
                {% for key, spec in specs.items() %}
                <tr>
                    <th>
                        {% for lang in spec.languages %}
                            <a name="language-{{lang}}"/>{{ lang }}<br/>
                        {% endfor %}
                    </th>
                    <td>
                        <a href="{{spec.urls.home}}">{{key}}</a>
                    </td>
                    <td>
                        <ul>
                        {% for pkgmgr, inst in spec.install.items() %}
                        <li>{{pkgmgr}}: <code>{{ inst }}</code></li>
                        {% endfor %}
                        </ul>
                    </td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        """
        ).render(specs=specs)
    )

### Notebook-Optimized Language Servers

These servers have support for notebooks and file editors. The `pyls` and
`r-languageserver` are well-tested, while `jedi` and `Julia` servers are
experimental. Please only install one language server per language (`jedi` or
`pyls` for Python - not both).

In [ ]:
nb_langs = ["pyls", "r-languageserver", "julia-language-server", "jedi-language-server"]
lang_server_table(
    {key: spec for key, spec in mgr.language_servers.items() if key in nb_langs}
)

The Scala language server (`metals`) is not currently auto-detected, but can be
configured as demonstrated in the
[configuration example](<./Configuring.ipynb#Example:-Scala-Language-Server-(metals)-integration>).

### NodeJS-based Language Servers

These servers have mostly been tested with file editors.

In [ ]:
lang_server_table(
    {
        key: spec
        for key, spec in mgr.language_servers.items()
        if "npm" in spec.get("install", [])
    }
)

NodeJS is a prerequisite for installation of any of the above language servers;
you can get it with:

```bash
conda install -c conda-forge nodejs
# or one of the following, as an administrator
choco install nodejs            # Windows with Chocolatey
sudo apt-get install nodejs     # Debian/Ubuntu
sudo brew install nodejs        # MacOS with Homebrew
sudo dnf install nodejs         # Fedora
sudo yum install nodejs         # RHEL/CentOS
```

#### Example: Getting All the NodeJS-based Language Servers

A number of language servers are built on the
[reference implementation](https://github.com/microsoft/vscode-languageserver-node),
powered by NodeJS. The most reliable place to install these is in a
`node_modules` in the directory where you launch `jupyter lab`.

For example, to install all the servers which are tested as part of
`jupyterlab-lsp`:

```bash
jlpm add --dev \
    bash-language-server \
    vscode-css-languageserver-bin \
    dockerfile-language-server-nodejs \
    vscode-html-languageserver-bin \
    javascript-typescript-langserver \
    vscode-json-languageserver-bin \
    yaml-language-server
```

This will create (or add to):

- `package.json` (check this in!)
- `yarn.lock` (check this in!)
- `node_modules/` (add to your VCS ignore file)

If you wish to install these someplace else, you may need to specify where you
install them with [extra_node_roots](./Configuring.ipynb#extra_node_roots).

### Other Scientific Languages

> These servers have been mostly tested with file editor.

In [ ]:
sci_langs = ["texlab"]
lang_server_table(
    {key: spec for key, spec in mgr.language_servers.items() if key in sci_langs}
)

#### Example: Getting a $\LaTeX$ stack

```bash
conda install -y conda-forge tectonic texlab chktex
```

This will install:

- `tectonic`, a cross-platform $\LaTeX$ processing tool
  - note, it will download a large number of packages when first executed
- `texlab`, a Language Server for `.tex` files that supports completion and
  reference navigation
- `chktex`, a `.tex` style checker

### Testing Languages

> These servers have been mostly tested with file editor.

In [ ]:
test_langs = ["robotframework_ls"]
lang_server_table(
    {key: spec for key, spec in mgr.language_servers.items() if key in test_langs}
)

In [ ]:
%%html
<style>
    table.langservers thead th {
        text-align: center !important;
    }

    table.langservers tbody td,
    table.langservers tbody th {
        text-align: left !important;
    }
</style>